In [1]:
import torch
import torchvision
from torchvision import transforms
from torch import nn
import numpy as np
import sys

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [4]:
# MNIST dataset

train_dataset = torchvision.datasets.MNIST(root='data', train=True,
                                           transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.MNIST(root='data', train=False,
                                          transform=transforms.ToTensor(), download=True)

batch_size = 10

# Data Loader

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                          shuffle=True)

In [5]:
# Convolutional neural network (two convolutional layers)

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
                            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                            nn.BatchNorm2d(16),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                            nn.BatchNorm2d(32),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(32*7*7, num_classes)
        self.drop_out = nn.Dropout(p=0.2)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
num_epochs = 1
num_classes = 10
learning_rate = 0.001

In [7]:
model = ConvNet(num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            sys.stdout.write('\rEpoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                            .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/1], Step [6000/6000], Loss: 0.0516

In [12]:
# Test the model
model.eval()  # eval mode will turn off the dropout; good to explicitly call when you test
all_labels = []
all_predicted = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.append(labels.cpu())  #for plotting
        all_predicted.append(predicted.cpu())

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'cnn.ckpt')

Test Accuracy of the model on the 10000 test images: 98.55 %


In [13]:
all_labels = np.array(all_labels).flatten()
all_predicted = np.array(all_predicted).flatten()

(all_labels == all_predicted).sum()

/tmp/ipykernel_194/292591539.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  all_labels = np.array(all_labels).flatten()
/tmp/ipykernel_194/292591539.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_labels = np.array(all_labels).flatten()
/tmp/ipykernel_194/292591539.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one 

AttributeError: 'bool' object has no attribute 'sum'

In [ ]:

# Display the confusion matrix as a heatmap
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

arr = confusion_matrix(all_labels, all_predicted)
class_names = np.arange(10)
df_cm = pd.DataFrame(arr, class_names, class_names)
plt.figure(figsize = (9,6))
sn.heatmap(df_cm, annot=True, fmt="d", cmap='BuGn')
plt.xlabel("prediction")
plt.ylabel("label (ground truth)")
plt.show();